In [428]:
# some useful websites for using pandas/yfinance:
# https://pandas.pydata.org/docs/user_guide/10min.html
# https://pypi.org/project/yfinance/

In [1]:
# import modules
# if you get a "module not found" error then run this command in a new code block and then try again: !pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# TODO: enter your ETF ticker, dates, and price interval
# set parameters for data pull
ticker = 'SPY'
start_date = '2000-01-01'
end_date = '2023-10-11'
interval = '1d'

# access data and display
df_etf = yf.download(ticker, start = start_date, end = end_date, interval= interval)
df_etf

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,148.250000,148.250000,143.875000,145.437500,94.262543,8164300
2000-01-04,143.531250,144.062500,139.640625,139.750000,90.576302,8089800
2000-01-05,139.937500,141.531250,137.250000,140.000000,90.738373,12177900
2000-01-06,139.625000,141.500000,137.750000,137.750000,89.280037,6227200
2000-01-07,140.312500,145.750000,140.062500,145.750000,94.465103,8066500
...,...,...,...,...,...,...
2023-10-04,422.070007,425.429993,420.559998,424.660004,424.660004,87453000
2023-10-05,424.359985,425.369995,421.170013,424.500000,424.500000,70142700
2023-10-06,421.970001,431.130005,420.600006,429.540009,429.540009,113202700


In [3]:
# consider the adjusted close column only
df_etf_ret = df_etf[['Adj Close']]

# calculate daily returns [(new / old) - 1]
df_etf_ret['ret'] = ((df_etf_ret['Adj Close'] / df_etf_ret['Adj Close'].shift()) - 1) * 100
df_etf_ret

C:\Users\andre\AppData\Local\Temp\ipykernel_23264\4257433714.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_etf_ret['ret'] = ((df_etf_ret['Adj Close'] / df_etf_ret['Adj Close'].shift()) - 1) * 100


,Adj Close,ret
Date,,
2000-01-03,94.262543,NaN
2000-01-04,90.576302,-3.910611
2000-01-05,90.738373,0.178933
2000-01-06,89.280037,-1.607188
2000-01-07,94.465103,5.807643
...,...,...
2023-10-04,424.660004,0.728197
2023-10-05,424.500000,-0.037678
2023-10-06,429.540009,1.187281


In [9]:
# Pull in Fama/French 3 Factors [Daily] csv file
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
df = pd.read_csv('RF.csv', skiprows=3, index_col=0, skipfooter=1)

# reformat index as a date, convert RF to decimal
df.index = pd.to_datetime(df.index, format='%Y%m%d', errors='coerce')
df['RF'] /= 100
df

C:\Users\andre\AppData\Local\Temp\ipykernel_23264\3126248487.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('RF.csv', skiprows=3, index_col=0, skipfooter=1)


,Mkt-RF,SMB,HML,RF
1926-07-01,0.10,-0.25,-0.27,0.00009
1926-07-02,0.45,-0.33,-0.06,0.00009
1926-07-06,0.17,0.30,-0.39,0.00009
1926-07-07,0.09,-0.58,0.02,0.00009
1926-07-08,0.21,-0.38,0.19,0.00009
...,...,...,...,...
2023-07-25,0.25,-0.04,-0.79,0.00022
2023-07-26,0.02,0.68,1.03,0.00022
2023-07-27,-0.74,-0.90,0.27,0.00022
2023-07-28,1.14,0.53,-0.33,0.00022


In [10]:
# merge risk-free rate and etf data 
df_result = pd.merge(df, df_etf_ret, left_index=True, right_index=True, how='inner').dropna()
df_result.drop(['Mkt-RF', 'SMB', 'HML'], axis=1, inplace=True)

# calculate excess returns
df_result['xs_ret'] = df_result['ret'] - df_result['RF']

# reset index and make date column
df_result.index.name = 'Date'
df_result

,RF,Adj Close,ret,xs_ret
Date,,,,
2000-01-04,0.00021,90.576302,-3.910611,-3.910821
2000-01-05,0.00021,90.738373,0.178933,0.178723
2000-01-06,0.00021,89.280037,-1.607188,-1.607398
2000-01-07,0.00021,94.465103,5.807643,5.807433
2000-01-10,0.00021,94.789192,0.343078,0.342868
...,...,...,...,...
2023-07-25,0.00022,453.839142,0.273004,0.272784
2023-07-26,0.00022,453.908905,0.015372,0.015152
2023-07-27,0.00022,450.899506,-0.662996,-0.663216


In [11]:
# pull in provided fomc data
df_fomc = pd.read_csv('FOMC_DATES.csv', index_col='Date')

# reformat dates
df_fomc.index = pd.to_datetime(df_fomc.index, format='%Y%m%d', errors='coerce')
df_fomc

,fomc
Date,
1978-01-17,1
1978-02-28,1
1978-03-21,1
1978-04-18,1
1978-05-05,1
...,...
2023-03-22,1
2023-05-03,1
2023-06-14,1


In [12]:
# merge excess returns and fomc dates
df_merged = pd.merge(df_result, df_fomc, left_index=True, right_index=True, how='left')
df_merged

,RF,Adj Close,ret,xs_ret,fomc
Date,,,,,
2000-01-04,0.00021,90.576302,-3.910611,-3.910821,NaN
2000-01-05,0.00021,90.738373,0.178933,0.178723,NaN
2000-01-06,0.00021,89.280037,-1.607188,-1.607398,NaN
2000-01-07,0.00021,94.465103,5.807643,5.807433,NaN
2000-01-10,0.00021,94.789192,0.343078,0.342868,NaN
...,...,...,...,...,...
2023-07-25,0.00022,453.839142,0.273004,0.272784,NaN
2023-07-26,0.00022,453.908905,0.015372,0.015152,1.0
2023-07-27,0.00022,450.899506,-0.662996,-0.663216,NaN


In [13]:
# TODO: implement your strategy here
# example strategy is to buy 3 days after the announcement and sell the following day
df_merged['buy'] = df_merged['fomc'].shift(3)
df_merged['sell'] = df_merged['fomc'].shift(4)
df_merged

,RF,Adj Close,ret,xs_ret,fomc,buy,sell
Date,,,,,,,
2000-01-04,0.00021,90.576302,-3.910611,-3.910821,NaN,NaN,NaN
2000-01-05,0.00021,90.738373,0.178933,0.178723,NaN,NaN,NaN
2000-01-06,0.00021,89.280037,-1.607188,-1.607398,NaN,NaN,NaN
2000-01-07,0.00021,94.465103,5.807643,5.807433,NaN,NaN,NaN
2000-01-10,0.00021,94.789192,0.343078,0.342868,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-07-25,0.00022,453.839142,0.273004,0.272784,NaN,NaN,NaN
2023-07-26,0.00022,453.908905,0.015372,0.015152,1.0,NaN,NaN
2023-07-27,0.00022,450.899506,-0.662996,-0.663216,NaN,NaN,NaN


In [14]:
# TODO: make new df for all days with money deployed
df_strategy = df_merged.query('sell == 1')
df_strategy

,RF,Adj Close,ret,xs_ret,fomc,buy,sell
Date,,,,,,,
2000-02-08,0.00022,93.533401,1.360842,1.360622,NaN,NaN,1.0
2000-03-16,0.00020,94.849922,4.671385,4.671185,NaN,NaN,1.0
2000-03-27,0.00020,98.725662,-1.058227,-1.058427,NaN,NaN,1.0
2000-05-22,0.00023,91.009560,-0.752914,-0.753144,NaN,NaN,1.0
2000-07-05,0.00024,94.195412,-1.803543,-1.803783,NaN,NaN,1.0
...,...,...,...,...,...,...,...
2022-12-20,0.00016,376.362640,0.136832,0.136672,NaN,NaN,1.0
2023-02-07,0.00018,410.632263,1.307856,1.307676,NaN,NaN,1.0
2023-03-28,0.00016,392.750580,-0.224466,-0.224626,NaN,NaN,1.0


In [15]:
# TODO: input number of days the trade is active for each FOMC meeting
# e.g. enter 6 if buying 1 day after FOMC and selling 7 days after FOMC
num_days_per_meeting = 1

# multiply by 8 (the average number of times the FOMC meets per year) for scaling
scale = num_days_per_meeting * 8

# calculate annualized Sharpe
mean = df_strategy['xs_ret'].mean() 
stdev = df_strategy['xs_ret'].std()
sharpe = (mean / stdev) * np.sqrt(scale)
print(sharpe)

0.23005814353287074
